# 2019 2C 1ra Oportunidad


Tokyo Telemessage se encuentra analizando la posibilidad de dar finalmente de baja su servicio de pager que mantiene desde la década de 1980. Los pagers son un dispositivo de mensajería que realiza un ‘beep’ al recibir un mensaje de texto de un número limitado de caracteres. Para poder enviar un mensaje, uno debe llamar a un número telefónico que representa al pager dejando el mensaje a una operadora. 
Para el analisis se cuenta con dos csv’s: clientes.csv del siguiente formato (id_pager, numero_telefono, codigo_de_area, fecha_creacion_cuenta, nombre_cliente, region, categoria_cliente) y eventos.csv (año,mes,dia,hora,minutos,segundos, id_pager, mensaje, numero_origen).

Como parte del análisis se desea responder:
¿Cuál es la región que aún tiene activos la mayor cantidad de pagers, entendiendo como activo aquellos que recibieron por lo menos un mensaje en el último mes?
¿Cuál es el porcentaje de pagers activos que solamente reciben mensajes únicamente de una persona (es decir, siempre desde el mismo número de origen)?
Utilizar pandas para responder las anteriores preguntas.

In [0]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [0]:
# armamos un data frame para usarlo como ejemplo de S&OP 500
tt_clients_data = {
        'id_pager': ['1','2','3','4','5'],
        'numero_telefono': ['1','2','3','4','5'],
        'codigo_de_area': ['11','11','11','22','22'],
        'fecha_creacion_cuenta': ['1983-01-01','1984-01-01','1984-01-01','1985-01-01','1986-01-01'],
        'nombre_cliente': ['Okabe', 'Kurisu', 'Mayuri','Suzuha', 'Daru'],
        'region': ['Akihabara', 'Akihabara', 'Akihabara', 'Kyoto', 'Kyoto'],
        'categoria_cliente': ['1','1','2','2','1']
}

tt_clients = pd.DataFrame(tt_clients_data, columns = tt_clients_data.keys())

In [0]:
# armamos un data frame para usarlo como ejemplo de S&OP 500
tt_msgs_data = {
        'anio': ['2020','2020','2020','2020','2020','2020','2020','2020','2020','2020'],
        'mes': ['3','3','3','3','4','4','4','4','4','4'],
        'dia': ['1','2','3','11','12','13','11','22','23','22'],
        'minutos': ['1','2','3','11','12','13','11','22','23','22'],
        'segundos': ['1','2','3','11','12','13','11','22','23','22'],
        'id_pager': ['2','1','2','1','5','1','2','1','2','1'],
        'mensaje': ["I'm the great mad scientist, Hououin Kyoma", "I am a scientist, I have to act on my own theory.", 
                    "It's an intellectual drink for the chosen ones", "People's feelings are memories that transcend time.",
                    "El Psy Kongroo", "Something must be wrong for you to use my actual name.",
                    "It's the choice of Steins Gate.", "You've pretty much figured it all out by now, right?",
                    "Sorry, The Zombie~~", "There was a scientific rationale for that!"],
        'numero_origen': ['1','2','1','2','1','1','2','1','2','1']
}

tt_msgs = pd.DataFrame(tt_msgs_data, columns = tt_msgs_data.keys())

In [22]:
tt_clients.head()

,id_pager,numero_telefono,codigo_de_area,fecha_creacion_cuenta,nombre_cliente,region,categoria_cliente
0,1,1,11,1983-01-01,Okabe,Akihabara,1
1,2,2,11,1984-01-01,Kurisu,Akihabara,1
2,3,3,11,1984-01-01,Mayuri,Akihabara,2
3,4,4,22,1985-01-01,Suzuha,Kyoto,2
4,5,5,22,1986-01-01,Daru,Kyoto,1


In [24]:
tt_msgs.head()

,anio,mes,dia,minutos,segundos,id_pager,mensaje,numero_origen
0,2020,3,1,1,1,2,"I'm the great mad scientist, Hououin Kyoma",1
1,2020,3,2,2,2,1,"I am a scientist, I have to act on my own theory.",2
2,2020,3,3,3,3,2,It's an intellectual drink for the chosen ones,1
3,2020,3,11,11,11,1,People's feelings are memories that transcend ...,2
4,2020,4,12,12,12,5,El Psy Kongroo,1


¿Cuál es la región que aún tiene activos la mayor cantidad de pagers, entendiendo como activo aquellos que recibieron por lo menos un mensaje en el último mes?

In [0]:
tt_totals = tt_msgs[((tt_msgs['anio'] == '2020') & (tt_msgs['mes'] == '4'))].groupby(['anio','mes','id_pager'])['dia'].agg(['count']).reset_index()  

In [35]:
merged = tt_totals[tt_totals['count'] >= 1].merge(tt_clients, how='inner')

,anio,mes,id_pager,count,numero_telefono,codigo_de_area,fecha_creacion_cuenta,nombre_cliente,region,categoria_cliente
0,2020,4,1,3,1,11,1983-01-01,Okabe,Akihabara,1
1,2020,4,2,2,2,11,1984-01-01,Kurisu,Akihabara,1
2,2020,4,5,1,5,22,1986-01-01,Daru,Kyoto,1


In [37]:
merged['region'].value_counts().nlargest(1)

Akihabara    2
Name: region, dtype: int64

¿Cuál es el porcentaje de pagers activos que solamente reciben mensajes únicamente de una persona (es decir, siempre desde el mismo número de origen)?

In [0]:
total_pagers_activos = len(tt_totals.index)

In [39]:
total_pagers_activos

3

In [0]:
tt_totals = tt_msgs[((tt_msgs['anio'] == '2020') & (tt_msgs['mes'] == '4'))].groupby(['anio','mes','numero_origen','id_pager'])['dia'].agg(['count']).reset_index()

In [46]:
tt_totals.head()

,anio,mes,numero_origen,id_pager,count
0,2020,4,1,1,3
1,2020,4,1,5,1
2,2020,4,2,2,2


In [0]:
only_one = tt_totals.groupby('numero_origen').filter(lambda x: len(x) == 1)

In [0]:
only_one_total = len(only_one.index)

In [50]:
print(only_one_total * 100 / total_pagers_activos)

33.333333333333336
